# Image alignment

In this task, you will have to solve two image alignment problems: channel processing and face alignment. You can get **10 points** implementing all the passed functions (7.5 for the first part and 2.5 for the second one).

# Image channels processing and alignment (7.5 points)

## Problem review

Sergey Prokudin-Gorsky was the first color photographer in Russia, who made the color portrait of Leo Tolstoy. Each of his photographs is three black-and-white photo plates, corresponding to red, green, and blue color channels. Currently, the collection of his pictures is situated in the U.S. Library of Congress, the altered versions have proliferated online. In this task, you should make a programme which will align the images from the Prokudin-Gorsky plates and learn the basic image processing methods.

*The input image and the result of the alignment:*
<img src="http://cdn1.savepice.ru/uploads/2017/7/31/8e68237bfd49026d137f59283db18b29-full.png">

In [ ]:
%pylab inline
import matplotlib.pyplot as plt 
import numpy as np

## Problem description

#### Input image loading

The input image is the set of 3 plates, corresponding to B, G, and R channels (top-down). You should implement the function $\tt{load}$\_$\tt{data}$ that reads the data and returns the list of images of plates.
$\tt{dir}$\_$\tt{name}$ is the path to the directory with plate images. If this directory is located in the same directory as this notebook, then default arguments can be used.

In [ ]:
from pathlib import Path
import cv2

def load_data(dir_name = 'plates'):
    """
    Loads the plate data
    
    Parameters
    -----------
    dir_name : str
        Path relative to location of this notebook
    
    Returns
    -------
    data : list
        The loaded data images. Each image is loaded as a numpy ndarray
    """
    
    # Sort as we would like to load the images in a sorted fashion
    files = sorted(Path('.').joinpath(dir_name).glob('*.png'))
    
    data = [cv2.imread(str(f), cv2.IMREAD_GRAYSCALE) for f in files]

    return data

plates = load_data()

The dataset is a list of 2-dimensional arrays.

In [ ]:
# The auxiliary function `visualize()` displays the images given as argument.
def visualize(imgs, format=None):
    plt.figure(figsize=(20, 40))
    for i, img in enumerate(imgs):
        if img.shape[0] == 3:
            img = img.transpose(1,2,0)
        plt_idx = i+1
        plt.subplot(3, 3, plt_idx)    
        plt.imshow(img, cmap=format)
    plt.show()

visualize(plates, 'gray')

#### The borders removal (1.5 points)
It is worth noting that there is a framing from all sides in most of the images. This framing can appreciably worsen the quality of channels alignment. Here, we suggest that you find the borders on the plates using Canny edge detector, and crop the images according to these edges. The example of using Canny detector implemented in skimage library can be found [here](http://scikit-image.org/docs/dev/auto_examples/edges/plot_canny.html).<br>

The borders can be removed in the following way:
* Apply Canny edge detector to the image.
* Find the rows and columns of the frame pixels. 
For example, in case of upper bound we will search for the row in the neighborhood of the upper edge of the image (e.g. 5% of its height). For each row let us count the number of edge pixels (obtained using Canny detector) it contains. Having these number let us find two maximums among them. Two rows corresponding to these maximums are edge rows. As there are two color changes in the frame (firstly, from light scanner background to the dark tape and then from the tape to the image), we need the second maximum that is further from the image border. The row corresponding to this maximum is the crop border. In order not to find two neighboring peaks, non-maximum suppression should be implemented: the rows next to the first maximum are set to zero, and after that, the second maximum is searched for.

#### Canny detector implementation (2.5 points)
You can write your own implementation of Canny edge detector to get extra points. <br>

Canny detection algorithm:
1. *Noise reduction.* To remove noise, the image is smoothed by Gaussian blur with the kernel of size $5 \times 5$ and $\sigma = 1.4$. Since the sum of the elements in the Gaussian kernel equals $1$, the kernel should be normalized before the convolution. <br><br>

2. *Calculating gradients.* When the image $I$ is smoothed, the derivatives $I_x$ and $I_y$ w.r.t. $x$ and $y$ are calculated. It can be implemented by convolving $I$ with Sobel kernels $K_x$ and $K_y$, respectively: 
$$ K_x = \begin{pmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{pmatrix}, K_y = \begin{pmatrix} 1 & 2 & 1 \\ 0 & 0 & 0 \\ -1 & -2 & -1 \end{pmatrix}. $$ 
Then, the magnitude $G$ and the slope $\theta$ of the gradient are calculated:
$$ |G| = \sqrt{I_x^2 + I_y^2}, $$
$$ \theta(x,y) = arctan\left(\frac{I_y}{I_x}\right)$$<br><br>

3. *Non-maximum suppression.* For each pixel find two neighbors (in the positive and negative gradient directions, supposing that each neighbor occupies the angle of $\pi /4$, and $0$ is the direction straight to the right). If the magnitude of the current pixel is greater than the magnitudes of the neighbors, nothing changes, otherwise, the magnitude of the current pixel is set to zero.<br><br>

4. *Double threshold.* The gradient magnitudes are compared with two specified threshold values, the first one is less than the second. The gradients that are smaller than the low threshold value are suppressed; the gradients higher than the high threshold value are marked as strong ones and the corresponding pixels are included in the final edge map. All the rest gradients are marked as weak ones and pixels corresponding to these gradients are considered in the next step.<br><br>

5. *Edge tracking by hysteresis.* Since a weak edge pixel caused from true edges will be connected to a strong edge pixel, pixel $w$ with weak gradient is marked as edge and included in the final edge map if and only if it is involved in the same blob (connected component) as some pixel $s$ with strong gradient. In other words, there should be a chain of neighbor weak pixels connecting $w$ and $s$ (the neighbors are 8 pixels around the considered one). You are welcome to make up and implement an algorithm that finds all the connected components of the gradient map considering each pixel only once.  After that, you can decide which pixels will be included in the final edge map (this algorithm should be single-pass, as well).

In [ ]:
# NOTE: This is the reference canny detection algorithm, see own implementation below

from  skimage.feature import canny

def Canny_detector(img):
    """ Your implementation instead of skimage """     
    return canny(img, sigma=1.4)

canny_imgs = []
for img in plates:
    canny_img = Canny_detector(img)
    canny_imgs.append(canny_img)
    
visualize(canny_imgs, 'gray')

In [ ]:
from scipy.signal import convolve2d

In [ ]:
def reduce_noise(img, sigma=1.4):
    """
    Input image is smoothed by Gaussian blur with sigma and a filter size of 5x5
    
    Parameters
    ----------
    img : np.array, shape (rows, cols)
        The image to reduce the noise of
    
    Returns
    -------
    noise_reduce : np.array, shape (rows, cols)
        The image with noise reduction
    """
    # The Gaussian is being centered around the center of the mesh
    # Therefore, we can make matrices which counts the distances from the center in the respective directions
    # We first make the distance array
    x_dist = [2, 1, 0, 1, 2]
    y_dist = [2, 1, 0, 1, 2]
    
    # And makes meshgrids out of these
    x, y = np.meshgrid(x_dist, y_dist)

    gaussian_kernel = (1/(2*np.pi*sigma**2))*np.exp(-(x**2 + y**2)/(2*sigma**2))
    # Normalization of the Gaussian kernel
    norm_gaussian_kernel = gaussian_kernel/gaussian_kernel.sum()
    
    # We convolve using mode='same' such that the input dimension equals output dimension
    noise_reduced = convolve2d(img, norm_gaussian_kernel, mode='same')
    
    return noise_reduced

In [ ]:
def get_gradients(img):
    """
    Returns the gradients with respect to x and y of the input image using Sobel kernels
    
    Parameters
    ----------
    img : np.array, shape (rows, cols)
        The image to take the derivative of
        
    Returns
    -------
    i_x : np.array, shape (rows, cols)
        The derivative of the image with respect to x
    i_y : np.array, shape (rows, cols)
        The derivative of the image with respect to y
    """
    
    k_x = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]])
    k_y = -k_x.T
    
    # We convolve using mode='same' such that the input dimension equals output dimension
    i_x = convolve2d(img, k_x, mode='same')
    i_y = convolve2d(img, k_y, mode='same')
    
    return i_x, i_y

In [ ]:
def non_maximum_suppression(g_abs, i_x, i_y):
    """
    Performs non maximum suppresses the image
    
    That is: 
    1. Finds the closest points in the positive and negative gradient direction
    2. If the point under investigation is less in magnitude than one of its two neighbors:
       Set value of point under investigation to zero
    
    Parameters
    ----------
    g_abs : np.array, shape (rows, cols)
        The magnitude of the gradient to take the non-maximum supression of
    i_x : np.array, shape (rows, cols)
        The derivative of the image with respect to x
    i_y : np.array, shape (rows, cols)
        The derivative of the image with respect to y
        
    Returns
    -------
    non_max_suppressed_g_abs : np.array, shape (rows, cols)
        The non-maximum suppressed image
    """
    
    non_max_suppressed_g_abs = g_abs.copy()
    
    row_inds, col_inds = g_abs.shape
    
    # Numpy is row major, so we loop over the columns first
    for x_ind in range(col_inds):
        for y_ind in range(row_inds):
            # First we find the value of the point under consideration
            # NOTE: We check the input g_abs, as the non_max_suppressed_g_abs may have changed
            cur_val = g_abs[y_ind, x_ind]
            
            # Next we find the closest points in positive and negative gradient direction
            # NOTE: Clipping values to 0 and 1, as long interactions are of less importances
            x_steps = np.round(i_x[y_ind, x_ind]).clip(0, 1).astype(int)
            y_steps = np.round(i_y[y_ind, x_ind]).clip(0, 1).astype(int)
            
            pos_x_index = (x_ind + x_steps).clip(0, col_inds-1)
            pos_y_index = (y_ind + y_steps).clip(0, row_inds-1)

            neg_x_index = (x_ind - x_steps).clip(0, col_inds-1)
            neg_y_index = (y_ind - y_steps).clip(0, row_inds-1)
            
            # Then we find the value at the positive and negative position
            # NOTE: We check the input g_abs, as the non_max_suppressed_g_abs may have changed
            pos_dir_val = g_abs[pos_y_index, pos_x_index]
            neg_dir_val = g_abs[neg_y_index, neg_x_index]
            
            if cur_val < pos_dir_val or cur_val < neg_dir_val:
                non_max_suppressed_g_abs[y_ind, x_ind] = 0

    return non_max_suppressed_g_abs

In [ ]:
def get_strong_and_weak_gradients(g_abs, high=90, low=80):
    """
    Returns the strong and weak gradients of an image
    
    A strong gradient is a gradient value which is above the high treshold.
    A weak gradient is a gradient value which is between the low and high treshold.
    Gradient values below the low threshold are neither.
    
    Notes
    -----
    The high treshold percentile is set high by default in order to capture the strongest gradients
    The low treshold percentile is set close to the high in order to suppress details
    
    Parameters
    ----------
    g_abs : np.array, shape (rows, cols)
        The magnitude of the gradient
    high : float
        The high treshold of the gradient (in percentiles)
    low : float
        The low threshold of the gradient (in percentiles)
        
    Returns
    -------
    strong_gradients : np.array, shape (rows, cols)
        Strong gradients of the image
    weak_gradients : np.array, shape (rows, cols)
        Weak gradients of the image    
    """
    
    high_tresh = np.percentile(g_abs, high)
    low_tresh = np.percentile(g_abs, low)
    
    strong_gradients = g_abs >= high_tresh
    weak_gradients = np.logical_and(g_abs >= low_tresh, g_abs < high_tresh)
    
    return strong_gradients, weak_gradients

In [ ]:
def edge_track_image(strong_gradients, weak_gradients):
    """
    Combines strong gradient pixels with weak edge pixels caused from true edges.
    
    A weak edge pixel is considered to be caused by a true edge if any of the eigth 
    neighbors contains a strong gradient.
    
    Parameters
    ----------
    strong_gradients : np.array, shape (rows, cols)
        Strong gradients of the image
    weak_gradients : np.array, shape (rows, cols)
        Weak gradients of the image       
    
    Returns
    -------
    canny_img : np.array, shape (rows, cols)
        The image containing the canny edges
    """
    
    canny_img = strong_gradients.copy().astype(int)
    
    row_inds, col_inds = weak_gradients.shape

    # Numpy is row major, so we loop over the columns first
    for x_ind in range(col_inds):
        for y_ind in range(row_inds):
            # NOTE: A point cannot simultaneously be a strong and weak gradient
            if weak_gradients[y_ind, x_ind] and\
               strong_gradients[y_ind-1:y_ind+1, x_ind-1:x_ind+1].any():
                canny_img[y_ind, x_ind] = 1
                
    return canny_img

In [ ]:
def Canny_detector(img):
    """
    Own implementation of the canny detection algorithm
    
    Parameters
    ----------
    img : np.array, shape (rows, cols)
        The image to find the Canny edges from
    
    Returns
    -------
    canny_img : np.array, shape (rows, cols)
        The image containing the canny edges
    """
    
    # 1. Noise reduction
    noise_reduced_img = reduce_noise(img)
    
    # 2. Calculating the gradients
    i_x, i_y = get_gradients(noise_reduced_img)
    # NOTE: One could use g_abs and theta for the non-maximum suppression,
    #       but using i_x and i_y uses suffices for our purposes
    g_abs = np.sqrt(i_x**2 + i_y**2)
    
    # 3. Non-maximum suppression
    suppressed_g = non_maximum_suppression(g_abs, i_x, i_y)
    
    # 4. Double threshold
    strong_gradients, weak_gradients = get_strong_and_weak_gradients(suppressed_g)
    
    # 5. Edge tracking by hysteresis
    canny_img = edge_track_image(strong_gradients, weak_gradients)
    
    return canny_img

canny_imgs = []
for img in plates:
    canny_img = Canny_detector(img)
    canny_imgs.append(canny_img)
    
visualize(canny_imgs, 'gray')

In [ ]:
#################################################################
# TODO: Implement the removal of the tape borders using         #
#       the output of Canny edge detector ("canny_img" list)    #
#################################################################


# * Find the rows and columns of the frame pixels. 
# For example, in case of upper bound we will search for the row in the neighborhood of the upper edge of the image
# (e.g. 5% of its height). 
# For each row let us count the number of edge pixels (obtained using Canny detector) it contains.
# Having these number let us find two maximums among them.
# Two rows corresponding to these maximums are edge rows. 
# As there are two color changes in the frame 
# (firstly, from light scanner background to the dark tape and then from the tape to the image),
# we need the second maximum that is further from the image border. 
# The row corresponding to this maximum is the crop border. 
# In order not to find two neighboring peaks, 
# non-maximum suppression should be implemented: 
# the rows next to the first maximum are set to zero, 
# and after that, the second maximum is searched for.

def remove_borders(img, canny_img):
    """ Your implementation instead of the following one"""   
    dx = int(img.shape[1] * 0.05) 
    return img[dx : -dx, dx : -dx]


cropped_imgs = []
#crop borders
for i, img in enumerate(plates):
    cropped_imgs.append(remove_borders(img, canny_imgs[i]))

visualize(cropped_imgs, 'gray')

#### Channels separation  (0.5 points)

The next step is to separate the image into three channels (B, G, R) and make one colored picture. To get channels, you can divide each plate into three equal parts.

In [ ]:
#################################################################
# TODO: implement the function impose_components transforming   #
#       cropped black-and-white images cropped_imgs             #
#       into the list of RGB images rgb_imgs                    #
#################################################################
def impose_components(img):
    """ Your implementation """   
    pass


rgb_imgs = []
for cropped_img in cropped_imgs:
    rgb_img = impose_components(cropped_img)
    rgb_imgs.append(rgb_img)

visualize(rgb_imgs)

#### Search for the best shift for channel alignment (1 point for metrics implementation + 2 points for channel alignment)

In order to align two images, we will shift one image relative to another within some limits (e.g. from $-15$ to $15$ pixels). For each shift, we can calculate some metrics in the overlap of the images. Depending on the metrics, the best shift is the one the metrics achieves the greatest or the smallest value for. We suggest that you implement two metrics and choose the one that allows to obtain the better alignment quality:

* *Mean squared error (MSE):*<br><br>
$$ MSE(I_1, I_2) = \dfrac{1}{w * h}\sum_{x,y}(I_1(x,y)-I_2(x,y))^2, $$<br> where *w, h* are width and height of the images, respectively. To find the optimal shift you should find the minimum MSE over all the shift values.
    <br><br>
* *Normalized cross-correlation (CC):*<br><br>
    $$
    I_1 \ast I_2 = \dfrac{\sum_{x,y}I_1(x,y)I_2(x,y)}{\sum_{x,y}I_1(x,y)\sum_{x,y}I_2(x,y)}.
    $$<br>
    To find the optimal shift you should find the maximum CC over all the shift values.

In [ ]:
#################################################################
# TODO: implement the functions mse и cor calculating           #
#       mean squared error and normalized cross-correlation     #
#       for two input images, respectively (1 point)            #
#################################################################
def mse(X, Y):
    """ Your implementation """     
    pass

def cor(X, Y):
    """ Your implementation """     
    pass

In [ ]:
#################################################################
# TODO: implement the algorithm to find the best channel        #
#       shift and the alignment. Apply this algorithm for       #
#       rgb_imgs processing and get the final list of colored   #
#       pictures. These images will be used for the evaluation  #
#       of the quality of the whole algorithm.  (2 points)      #
#                                                               #
#       You can use the following interface or write your own.  #
#################################################################

def get_best_shift(channel1, channel2):
    """ Your implementation """     
    pass


def get_best_image(rgb_img):
    """ Your implementation """     
    pass



final_imgs = []
for img in rgb_imgs:
    final_img = get_best_image(img)
    final_imgs.append(final_img)

visualize(final_imgs)

# Face Alignment (2.5 points)

In this task, you have to implement face normalization and alignment. Most of the face images deceptively seem to be aligned, but since many face recognition algorithms are very sensitive to shifts and rotations, we need not only to find a face on the image but also normalize it. Besides, the neural networks usually used for recognition have fixed input size, so, the normalized face images should be resized as well.

There are six images of faces you have to normalize. In addition, you have the coordinates of the eyes in each of the pictures. You have to rotate the image so that the eyes are on the same height, crop the square box containing the face and transform it to the size $224\times 224.$ The eyes should be located symmetrically and in the middle of the image (on the height).

Here is an example of how the transformation should look like.

<img src = "https://cdn1.savepice.ru/uploads/2017/12/13/286e475ef7a4f4e59005bcf7de78742f-full.jpg">

#### Get data
You get the images and corresponding eyes coordinates for each person. You should implement the  function $\tt{load}$\_$\tt{faces}$\_$\tt{and}$\_$\tt{eyes}$ that reads the data and returns two dictionaries: the dictionary of images and the dictionary of eyes coordinates. Eyes coordinates is a list of two tuples $[(x_1,y_1),(x_2,y_2)]$.
Both dictionaries should have filenames as the keys.

$\tt{dir}$\_$\tt{name}$ is the path to the directory with face images, $\tt{eye}$\_$\tt{path}$ is the path to .pickle file with eyes coordinates. If these directory and file are located in the same directory as this notebook, then default arguments can be used.

In [ ]:
def load_faces_and_eyes(dir_name = 'faces_imgs', eye_path = './eyes.pickle'):
    """ Your implementation """     
    pass
    
    
faces, eyes = load_faces_and_eyes()

Here is how the input images look like:

In [ ]:
visualize(faces.values())

You may make the transformation using your own algorithm or by the following steps:
1. Find the angle between the segment connecting two eyes and horizontal line;
2. Rotate the image;
3. Find the coordinates of the eyes on the rotated image
4. Find the width and height of the box containing the face depending on the eyes coordinates
5. Crop the box and resize it to $224\times224$

In [ ]:
#################################################################
# TODO: implement the function transform_face that rotates      #
#       the image so that the eyes have equal ordinate,         #
#       crops the square box containing face and resizes it.    #
#       You can use methods from skimage library if you need.   #
#       (2.5 points)                                              #
#################################################################

def transform_face(image, eyes):
    """ Your implementation """     
    return image

In [ ]:
transformed_imgs = []
for i in faces:
    img = faces[i]
    eye = eyes[i]
    transformed = transform_face(img, eye)
    transformed_imgs.append(transformed)
    
visualize(transformed_imgs)